In [14]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_9.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    return max_corr if max_corr >= threshold else 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename):
    submatrix = pd.DataFrame(index=columns, columns=columns)
    for col1, col2 in product(columns, repeat=2):
        submatrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    submatrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix{filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


#filename_sub = "9_sub_mat_3.csv"
#compute_submatrix(data, group_3, filename_sub)

filename_cross = "9_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross)


submatrix9_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,0.396759,0.39461,0.364012,0.400748,0.469506,0.560667,0.414772,0.316517,0.48584,0.493875,...,0.306691,0.457393,0.445643,0.431873,0.433344,0.415912,0.521606,0.504497,0.455837,0.51978
21085.0,0.399809,0.377998,0.26103,0.327714,0.491327,0.507233,0.388156,0.344662,0.553509,0.458647,...,0.226097,0.455254,0.345631,0.280129,0.316482,0.370754,0.390057,0.361789,0.344195,0.309883
21087.0,0.425053,0.376053,0.306509,0.312484,0.389053,0.402211,0.361232,0.339125,0.460569,0.45477,...,0.300776,0.283822,0.286607,0.362835,0.415718,0.332019,0.292139,0.475861,0.356087,0.245832
21089.0,0.445646,0.392299,0.225864,0.264238,0.39147,0.535042,0.449708,0.330791,0.506453,0.459838,...,0.367604,0.489324,0.361154,0.429575,0.388854,0.382037,0.419526,0.458271,0.434451,0.566886
21091.0,0.183309,0.150083,0.07638,0.093737,0.474873,0.361873,0.364909,0.148155,0.411953,0.372571,...,0.141095,0.376374,0.354796,0.170106,0.193886,0.166962,0.545931,0.361672,0.193116,0.424007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,0.783095,0.771729,0.34705,0.355621,0.308564,0.609956,0.474464,0.401445,0.560463,0.535801,...,0.605719,0.330137,0.357159,0.635105,0.713417,0.641161,0.229776,0.487446,0.697112,0.127076
38091.0,0.612732,0.582287,0.398643,0.378602,0.30402,0.460533,0.432629,0.350135,0.412779,0.490522,...,0.57508,0.417438,0.347984,0.587174,0.596678,0.524441,0.26207,0.444822,0.56466,0.132535
38093.0,0.746485,0.779737,0.44688,0.517871,0.422157,0.564094,0.446231,0.43396,0.467166,0.559037,...,0.648282,0.486916,0.368807,0.602627,0.591756,0.625228,0.272942,0.503005,0.581518,0.158328
38095.0,0.491176,0.540283,0.318272,0.348352,0.416588,0.522916,0.48943,0.314245,0.38813,0.604203,...,0.470545,0.413186,0.386785,0.535415,0.484614,0.518606,0.28093,0.471494,0.509646,0.225916
